# 异常处理

程序中的错误分为3种:

+ 用户错误
    
    预期会发生的错误,用户的特定操作就会导致特定的错误,比如注册邮箱时填入的内容没有"@"这种.这种错误通常应该是程序主体就要处理的.通过返回特定的错误码进行提示.
    
    
+ 运行时错误

    运行时错误从来都是非预期的,表明程序出了bug,程序必须要结束.通常需要使用断言来捕获这种错误
    
    
+ 异常

    介于洪湖错误和程序bug之间,异常可能是比较罕见的错误,比如算数运算时候的溢出情况.通常异常不返回错误码,而是由恢复代码处理,类似于非局部的goto.

C语言没有异常,更不要提异常处理了,一般来说是通过自己规定的返回码来标识异常的.通常使用标准库`errno.h`配合标准库`string.h`中的函数`strerror`和标准库`stdio.h`中的`perror`用于打印异常信息.

`errno.h`是C标准库中的一个检查错误的头文件,它的内部有一个全局变量`errno`.它可以被赋值,而C库中的每个模块都会包含这个errno.h头文件,那么errno这个可以被赋值的宏,就相当于整个C标准库的全局变量,如果在使用某个模块的函数过程中发生了错误,这个errno宏就会在函数内部被赋予一个有含义的值,相当于一个错误代码,并且无论库的哪个模块或库函数都不会把errno设置为零.所以我们在使用这个模块的时候只需要在一个库函数调用前将errno设置为零,然后在调用之后检查errno的值即可知道这个被调用的库函数有没有发生错误.关于errno.h的使用技巧可以[这篇文章](http://c.biancheng.net/c/errno/)


一个简单的例子如下:

In [1]:
#include <stdio.h>
#include <errno.h>
#include <string.h>

int main() {
    errno=0;
    FILE *fp = fopen("test.txt","r");
    if(errno!=0)
    {
        printf("errno值： %d\n",errno);
        printf("错误信息： %s\n",strerror(errno));
        perror(strerror(errno));
    }
}

errno值： 2
错误信息： No such file or directory


No such file or directory: No such file or directory


比较常见的ERROR定义如下:

```Cpp
#define EPERM        1  /* Operation not permitted */
#define ENOENT       2  /* No such file or directory */
#define ESRCH        3  /* No such process */
#define EINTR        4  /* Interrupted system call */
#define EIO      5  /* I/O error */
#define ENXIO        6  /* No such device or address */
#define E2BIG        7  /* Argument list too long */
#define ENOEXEC      8  /* Exec format error */
#define EBADF        9  /* Bad file number */
#define ECHILD      10  /* No child processes */
#define EAGAIN      11  /* Try again */
#define ENOMEM      12  /* Out of memory */
#define EACCES      13  /* Permission denied */
#define EFAULT      14  /* Bad address */
#define ENOTBLK     15  /* Block device required */
#define EBUSY       16  /* Device or resource busy */
#define EEXIST      17  /* File exists */
#define EXDEV       18  /* Cross-device link */
#define ENODEV      19  /* No such device */
#define ENOTDIR     20  /* Not a directory */
#define EISDIR      21  /* Is a directory */
#define EINVAL      22  /* Invalid argument */
#define ENFILE      23  /* File table overflow */
#define EMFILE      24  /* Too many open files */
#define ENOTTY      25  /* Not a typewriter */
#define ETXTBSY     26  /* Text file busy */
#define EFBIG       27  /* File too large */
#define ENOSPC      28  /* No space left on device */
#define ESPIPE      29  /* Illegal seek */
#define EROFS       30  /* Read-only file system */
#define EMLINK      31  /* Too many links */
#define EPIPE       32  /* Broken pipe */
#define EDOM        33  /* Math argument out of domain of func */
#define ERANGE      34  /* Math result not representable */
```

C语言中也往往用这种方式自定义模块的宏,自定义一个模块级的全局变量并结合一个错误枚举以及从枚举值转换为字面量的函数.就可以实现上面的错误处理方式了.

明显的这种方式在处理异常时是不会终止程序的,如果需要终止程序,我们就得使用标准库`stdlib.h`的`abort()`方法

In [3]:
#include <stdio.h>
#include <errno.h>
#include <string.h>
#include <stdlib.h>

int main() {
    errno=0;
    FILE *fp = fopen("test.txt","r");
    if(errno!=0){
        printf("errno值： %d\n",errno);
        printf("错误信息： %s\n",strerror(errno));
        perror(strerror(errno));
        abort();
    }
    for(int i=0;i<5;i++){
        printf("%d\n",i);
    }
}

No such file or directory: No such file or directory


errno值： 2
错误信息： No such file or directory


[C kernel] Executable exited with code -6

### 断言

C语言标准库有头文件`assert.h`将`assert(e)`定义为宏,`assert(e)`会计算表达式e的值,如果e为0则向标准错误`stderr`写出诊断信息,并调用标准库函数abort退出程序执行.诊断信息包含失败的断言文本和断言出现的坐标,该信息的格式是具体实现定义的.`assert(0)`可以用户指明"不可能发生"的情况,当然如下也对:

In [4]:
#include <stdio.h>
#include <errno.h>
#include <string.h>
#include <assert.h>

int main() {
    assert(!"ptr==NULL -- can't happen");
}

Assertion failed: (!"ptr==NULL -- can't happen"), function main, file /var/folders/2w/71b83qh117q3sjhm8xgnr7w00000gn/T/tmp11prdxto.c, line 7.
[C kernel] Executable exited with code -6

## 另一种方案

另一种方案在"C语言接口与实现"一书中有提到--使用标准库提供了`setjmp`和`longjmp`建立结构化异常处理,具体来说,`setjmp`实例化一个处理程序,`longjmp`引发一个异常

### try-except代码块

try-except代码块在现代编程语言中几乎是标配,在实践中也被证明非常有用.然而C中没有."C语言接口与实现"一书中有一个使用宏的实现,接下来会写一个库专门用于实现try-except代码块.不过有几个受限的使用形式:

+ `TRY-END_TRY`  

    相当于`{xxx}`语句不变只是引入一个新的作用域
    
+ `TRY-ELSE-END_TRY`  

    相当于python中的`try-except Exception`,只要有异常就处理,不管是啥异常
    
+ `TRY-EXCEPT-...-END_TRY` 
    
    相当于python中的`try-except E_1-except E_2-...`
    
    
+ `TRY-EXCEPT-...-ELSE-END_TRY` 
    
    注意这与python中不同,相当于python中的`try-except E_1-...- except Exception`
    
    
+ `TRY-EXCEPT-FINALLY-END_TRY`

    相当于python中的`try-except E_1-finally`无论是否有异常都会执行FINALLY,这个用法可以像python中的with语句一样提供一个上下文.
    
+ `TRY-EXCEPT-ELSE-FINALLY-END_TRY`

    相当于python中的`try-except E_1-except Exception-finally`无论是否有异常都会执行FINALLY,这个用法可以像python中的with语句一样提供一个上下文.

+ 如果`TRY`语句中需要执行返回,那么必须使用宏`RETURN`返回.


"C语言接口与实现"一书中提供的这组宏显然还是比较粗糙而且脆弱的,宏毕竟不会检查运行时错误,但一般用用足够.C语言中并不提倡大量使用异常,设计算法的时候这组宏也应当被谨慎的使用.

这个模块在代码中有收录,它与下面的断言收录在一起.

### C语言接口与实现中的断言


"C语言接口与实现"一书中提供了一个标准库"assert.h"的替代,用它的好处是如果断言失败不会向stderr打印消息,也不会跳出程序,也不会提供表达式e的文本,而是抛出上文中定义的`Assert_Failed`异常.